In [9]:
using Statistics
using LinearAlgebra
using FFTW
using Dates
using TimerOutputs
using DelimitedFiles # For CSV output
using Printf        # For formatted printing
using XLSX          # For writing to Excel files
using DataFrames


# -----------------------------
# Precision setup
# -----------------------------
const FullFloat = Float64
const ReduFloat = Float32

# -----------------------------
# Build spatial grid and derivative matrix
# -----------------------------
function setup_problem(Nx::Int, L::FullFloat, FullFloatType::DataType, ReduFloatType::DataType)
    x = LinRange(FullFloat(0), L, Nx + 1)[1:end-1]
    dx = x[2] - x[1]

    # Define k_vals in un-shifted order
    k_vals = collect(FullFloatType, vcat(0:((Nx-1) ÷ 2), -((Nx-1) ÷ 2):-1))

    # Full precision derivative matrix
    Iden_high = Matrix{FullFloatType}(I, Nx, Nx)
    K_high = Diagonal(k_vals)
    Dx_high = real(ifft((im * one(FullFloatType)) * K_high * fft(Iden_high, 1), 1))

    # Low precision derivative matrix
    Iden_low = Matrix{ReduFloatType}(I, Nx, Nx)
    K_low = Diagonal(ReduFloatType.(k_vals))
    Dx_low = real(ifft((im * one(ReduFloatType)) * K_low * fft(Iden_low, 1), 1))
    Dx_low = ReduFloatType.(Dx_low) 
    
    return x, dx, Dx_high, Dx_low
end


# -----------------------------
# Newton solver for one step (low precision)
# -----------------------------
function newton_solve(f::Function, x0::Vector{ReduFloat}, Dx_low::Matrix{ReduFloat}, dt_low::ReduFloat; maxiter::Int = 100)
    tol = 1e-5
    x = copy(x0)
    Id = Matrix{ReduFloat}(I, length(x0), length(x0))
    J = similar(Id)
    fx = similar(x)
    h2 = dt_low / ReduFloat(2)
    for iter in 1:maxiter
        fx .= f(x)
        res = norm(fx, Inf)
        if res < tol
            return x, res # Return converged solution and iteration count
        end
        J .= Id .+ (h2) .* (Dx_low * Diagonal(x))
        x .= x .- J \ fx # Corrected update step
    end
    # Return the non-converged solution and max iterations
    return x, norm(fx, Inf)
end


# -----------------------------
# Time integration using Newton's method (Mixed Precision)
# -----------------------------
function run_newton_mp(x::Vector{FullFloat}, Dx_high::Matrix{FullFloat}, Dx_low::Matrix{ReduFloat}; 
                       Nt::Int, dt::FullFloat)
    u = sin.(x)
    ulow = ReduFloat.(u)
    dt_low = ReduFloat(dt)
    y_high = similar(u)

    for i in 1:Nt
        ulow .= ReduFloat.(u)
        f_low = y -> y .- ulow .- (dt_low / ReduFloat(2)) .* (ReduFloat(-0.5) .* Dx_low * (y .^ ReduFloat(2)))
        y_low, res = newton_solve(f_low, ulow, Dx_low, dt_low)
        y_high .= FullFloat.(y_low)
        u .= u .+ dt .* (FullFloat(-0.5) .* Dx_high * (y_high.^FullFloat(2)))
        if any(isnan, u)
            println("Warning: NaN detected in Mixed-Precision Newton run.")
            break   
        end
    end
    return u
end

# -----------------------------
# Broyden solver for one time step
# -----------------------------
function broyden_solve(y0::Vector{FullFloat}, un::Vector{FullFloat}, dt::FullFloat, Dx::Matrix{FullFloat}, 
                       Ji::Matrix{FullFloat}, ResTol::FullFloat; good::Bool=true, maxiter::Int=1)
    y = copy(y0)
    fun_curr =  y .- un .+ (dt / 4) .* (Dx * (y.^ 2))
    # To avoid extra allocation
    fun_old = similar(fun_curr)
    s = similar(y)
    b = similar(y')
    dy = similar(y)
    
    
    for ia in 1:maxiter
        fun_old .= fun_curr
        s .= -Ji * fun_old
        y .= y .+ s
        fun_curr .= y .- un .+ (dt / 4) .* (Dx * (y.^ 2))  
        dy .= fun_curr - fun_old
        b .= good ? (s' * Ji) : dy'
        denom = (b * dy)
        if abs(denom) < 1e-12
            return y, Ji
        end
        Ji .= Ji + ((s - Ji * dy) * b) / (denom)
        res = norm(fun_curr, Inf)
        if res < ResTol 
            return y,  Ji
        end
    end
    return y,  Ji
end

# -----------------------------
# Broyden solver for semi-implicit midpoint rule
# -----------------------------
function run_broyden_midpoint(x::Vector{FullFloat}, Dx_high::Matrix{FullFloat}, Dx_low::Matrix{ReduFloat}; 
                              Nx::Int, Nt::Int, dt::FullFloat, ResTol::FullFloat)

    dt_low = ReduFloat(dt)
    un = sin.(x)
    ulow = ReduFloat.(un)
    Iden_low = Matrix{ReduFloat}(I, Nx, Nx)
    J = Iden_low .+ (ReduFloat(0.5) * dt_low) .* (Dx_low .* ulow')
    Ji = FullFloat.(inv(J))
    good = true
    f1 = similar(un)
    y = similar(un)
    fx = similar(un)

    for i in 1:Nt

        # Low precision implicit solve
        ulow .= ReduFloat.(un)      
        f_low = y -> y .- ulow .+ (dt_low / ReduFloat(2)) .* (ReduFloat(0.5) .* Dx_low * (y .^ ReduFloat(2)))
        y_low, res = newton_solve(f_low, ulow, Dx_low, dt_low)
        # Corrections
        y .= FullFloat.(y_low)
        #if res > ResTol
        y, Ji = broyden_solve(y, un, dt, Dx_high, Ji, ResTol; good=good)
        #end
        # Explicit stage
        f1 .= -Dx_high * (0.5 .* (y .^ FullFloat(2)))
        un .= un .+ dt .* f1

        if any(isnan, un)
            println("Warning: NaN detected in Broyden run.")
            break
        end
    end

    return un
end


# -----------------------------
# NEWTON SOLVER for IMR (Full Precision)
# -----------------------------
function run_newton_full_precision(x::Vector{FullFloat}, Dx_high::Matrix{FullFloat};
                                   Nx::Int, Nt::Int, dt::FullFloat,
                                   maxiter::Int=100)
    
    un = sin.(x)
    Iden = Matrix{FullFloat}(I, Nx, Nx)
    J = similar(Iden)
    ResTol = 1e-10
    s = similar(un)
    f1 = similar(un)
    y = similar(un)
    fx = similar(un)

    for i in 1:Nt
        y .= un
        for ia in 1:maxiter
            fx .= y .- un .+ (dt / 4) .* (Dx_high * (y.^ 2))
            res = norm(fx, Inf)
            if res < ResTol
                break
            end
            J .= Iden .+ (dt / 2) .* (Dx_high * Diagonal(y))
            s .= - (J \ fx)
            y .= y .+ s
        end
        

        f1 .= -Dx_high * (0.5 .* (y .^ 2))
        un .= un .+ dt .* f1
        
        if any(isnan, un)
            println("Warning: NaN detected in full-precision Newton.")
            break
        end
    end
    
    return un
end

function rk4_step(u, D, dt)
    
    k1 = -0.5*D*(u.^2)
    k2 = -0.5*D*(u + 0.5 * dt * k1).^2
    k3 = -0.5*D*(u + 0.5 * dt * k2).^2
    k4 = -0.5*D*(u + dt * k3).^2
    return u + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)
end

function run_ref(x::Vector{FullFloat}, Dx::Matrix{FullFloat}; Nt::Int, dt::FullFloat)
    un = sin.(x)
    for i in 1:Nt
        un .= rk4_step(un, Dx, dt)
    end
    return un
end





run_ref (generic function with 1 method)

In [11]:
# -----------------------------
# Main Experiment Function
# -----------------------------
function run_experiment()
    # --- Experiment Parameters ---
    Nx_values = [201, 301]
    Nt_values = [7000, 14000]
    tFinal = 0.7
    L = 2 * pi
    NUM_RUNS = 10
    Nt_ref = 10000
    FullFloat = Float64
    ReduFloat = Float32


    # --- DataFrame Initialization ---
    # Added columns for the reference Newton solver results.
    results_df = DataFrame(
        Nx = Int[],
        Nt = Int[],
        dx = Float64[],
        dt = Float64[],
        CFL = Float64[],
        # FP
        Time_FP = Float64[],
        Error_FP= Float64[],
        Time_MP = Float64[],
        Error_MP = Float64[],
        Speedup_MP = Float64[],
        # MP with corrections
        Time_B_MP = Float64[],
        Error_B_MP= Float64[],
        Speedup_B_MP = Float64[],

    )

    println("--- Starting Benchmark ---")
    println("Averaging timings over $NUM_RUNS runs for each solver.")
    println("High precision (FullFloat): ", FullFloat)
    println("Low precision (ReduFloat): ", ReduFloat)
    println("Reference Nt for error calculation: $Nt_ref")
    println("-"^60)

    for Nx in Nx_values
        println("Setting up for Nx = $Nx...")
        x_linrange, dx, Dx_high, Dx_low = setup_problem(Nx, L, FullFloat, ReduFloat)
        x = collect(x_linrange)

        println(" Generating high-accuracy reference solution...")
        dt_ref = tFinal / Nt_ref
        
        # Time the reference solution generation and capture its results
        ref_time = 0.0
        ref_result = nothing
        ref_time = @elapsed begin
            u_ref = run_ref(x, Dx_high; Nt = Nt_ref, dt = dt_ref)
        end
        @printf " Reference solution generated in %.4fs .\n" ref_time

        for Nt in Nt_values
            dt = tFinal / Nt
            cfl = dt / dx
            @printf " Running case: Nx=%d, Nt=%d, CFL=%.4f.\n" Nx Nt cfl 

            u_fp = u_mp = u_mp_b = nothing

            # Full precision
            total_time_fp = fill(0.0, NUM_RUNS)
            for i in 1:NUM_RUNS
                total_time_fp[i] = @elapsed begin
                     u_fp = run_newton_full_precision(x, Dx_high; Nx=Nx, Nt=Nt, dt=dt)
                end
            end
            time_fp = mean(total_time_fp[2:end])
            error_fp = norm(u_fp .- u_ref, Inf)


            # Mixed Precision with no corrections
            total_time_mp = fill(0.0, NUM_RUNS)
            for i in 1:NUM_RUNS
                total_time_mp[i] = @elapsed begin
                    u_mp = run_newton_mp(x, Dx_high, Dx_low; Nt = Nt, dt = dt)
                end
            end
            time_mp = mean(total_time_mp[2:end])
            error_mp = norm(u_mp .- u_ref, Inf)


            # Mixed Precision with corrections
            total_time_mp_b = fill(0.0, NUM_RUNS)
            for i in 1:NUM_RUNS
                total_time_mp_b[i] = @elapsed begin
                    u_mp_b = run_broyden_midpoint(x, Dx_high, Dx_low; 
                        Nx = Nx, Nt = Nt, dt = dt, ResTol=1e-12)
                end
            end
            time_mp_b = mean(total_time_mp_b[2:end])
            error_mp_b = norm(u_mp_b .- u_ref, Inf)

            speedup_mp = time_fp / time_mp
            speedup_mp_b = time_fp / time_mp_b

            # --- Push a new row to the DataFrame with reference solver data ---
            new_row = (
                Nx = Nx, Nt = Nt, dx = dx, dt = dt, CFL = cfl, Time_FP =  time_fp, Error_FP = error_fp, 
                Time_MP = time_mp, Error_MP = error_mp, Speedup_MP = speedup_mp,  
                Time_B_MP = time_mp_b, Error_B_MP = error_mp_b,  Speedup_B_MP =  speedup_mp_b
            )
            push!(results_df, new_row)
        end
        println("-"^60)
    end

    println("\n--- Benchmark Results Summary (from DataFrame) ---")

    # --- Generate a unique filename using a timestamp ---
    timestamp = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    # output_filename = "benchmark_results_$(timestamp).xlsx"

    # Make sure this file doesn't already exist in your directory
    output_filename = "IMR_BRG_1e4.xlsx"

    # --- Export the DataFrame ---
    # Now you don't need `overwrite=true` because the filename is always new.
    XLSX.writetable(output_filename, results_df, sheetname="Results")

    println("\nBenchmark finished.")
    println("Successfully exported results to '$(output_filename)'.")

end

run_experiment (generic function with 1 method)

In [12]:
# -----------------------------
# Run the experiment
# -----------------------------
run_experiment()

--- Starting Benchmark ---
Averaging timings over 10 runs for each solver.
High precision (FullFloat): Float64
Low precision (ReduFloat): Float32
Reference Nt for error calculation: 10000
------------------------------------------------------------
Setting up for Nx = 201...
 Generating high-accuracy reference solution...
 Reference solution generated in 0.5325s .
 Running case: Nx=201, Nt=7000, CFL=0.0032.
 Running case: Nx=201, Nt=14000, CFL=0.0016.
------------------------------------------------------------
Setting up for Nx = 301...
 Generating high-accuracy reference solution...
 Reference solution generated in 0.5581s .
 Running case: Nx=301, Nt=7000, CFL=0.0048.
 Running case: Nx=301, Nt=14000, CFL=0.0024.
------------------------------------------------------------

--- Benchmark Results Summary (from DataFrame) ---

Benchmark finished.
Successfully exported results to 'IMR_BRG_1e4.xlsx'.
